<a href="https://colab.research.google.com/github/LamaAlshabani/Tuwaiq_Academy/blob/main/spark/Colab/PySpark_On_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Spark Stream to get the top twitter hashtags related a keyword or a hashtag

In [24]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from pyspark import SparkContext # https://www.tutorialspoint.com/pyspark/pyspark_sparkcontext.htm
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from collections import namedtuple # https://www.geeksforgeeks.org/namedtuple-in-python/

import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [28]:
# Can only run this once. 
# restart your kernel for any errors.
sc = SparkContext()

ValueError: ignored

In [ ]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)
socket_stream = ssc.socketTextStream("127.0.0.1", 5553)
lines = socket_stream.window( 20 )

In [ ]:
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list # lambda is a function # (lambda arguments : expression)
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) #  Count and remove duplicates. #https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.RDD.reduceByKey.html
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts them in Descending order in a DataFrame.
  .limit(10).registerTempTable("tweets") ) ) #Registers this DataFrame as a temporary table using the given name)

__________
### Before start next ssc.start() you should run TweetListener.py in the terminal
```
python TweetListener.py
```
__________

In [ ]:
!wget https://raw.githubusercontent.com/Ruqyai/Tuwaiq_Academy/main/spark/TweetsListener.py &> /dev/null

In [ ]:
#run the script/API in the background
import subprocess
subprocess.Popen(["python", "/content/TweetsListener.py"]) 

In [ ]:
ssc.start()

In [ ]:
count = 0
while count < 10:
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    plt.show()
    count = count + 1

In [ ]:
ssc.stop()

# Assignment #2

Do any enhancement in the code or change anything 

In [ ]:
#When I try to change anything, I get an error